# Counting Squarefree numbers

A positive integer that is not divisible by the square of any prime is called **squarefree**. For example, $5$ is squarefree, but $20$ is not, since $2^2 \mid 20$.

Let $\mathcal{Q}(x)$ denote the number of squarefree integers not exceeding $x$.

In this notebook, we will cover methods to count squarefree numbers.

In [1]:
import numpy as np

from typing import Dict

from core import integer_sqrt

## Brute force counting

It is simple to determine whether an integer is squarefree based on its prime factorization. We can obtain the factorizations of all integers $\leq x$ by a simple sieve.

In [2]:
def factorizations(n: int) -> Dict[int, Dict[int, int]]:
    """Returns dict containing factorizations of integers k, 2 <= k < n.
    
    Parameters:
        n: int
    
    Returns:
        Dict[int, Dict[int, int]]: Map containing factorizations for each
        integer 2 <= k < n. Each factorization is a map prime -> exponent.
    """
    block = list(range(n))
    factors = {k: {} for k in range(2, n)}
    
    for d in range(2, n):
        if block[d] == 1:
            continue
            
        for multiple in range(d, n, d):
            factors[multiple][d] = 0
            
            while block[multiple] % d == 0:
                block[multiple] //= d
                factors[multiple][d] += 1
    
    return factors


# Simple sanity test on small values
for (k, factorization) in factorizations(1000).items():
    product = 1
    for (p, e) in factorization.items():
        product *= p**e
    assert k == product

In [3]:
# Prime factorizations of integers < 20, given as prime: exponent dicts
factorizations(20)

{2: {2: 1},
 3: {3: 1},
 4: {2: 2},
 5: {5: 1},
 6: {2: 1, 3: 1},
 7: {7: 1},
 8: {2: 3},
 9: {3: 2},
 10: {2: 1, 5: 1},
 11: {11: 1},
 12: {2: 2, 3: 1},
 13: {13: 1},
 14: {2: 1, 7: 1},
 15: {3: 1, 5: 1},
 16: {2: 4},
 17: {17: 1},
 18: {2: 1, 3: 2},
 19: {19: 1}}

Given the factorizations, it is simple to detect squarefree integers.

In [4]:
def is_squarefree(factorization: Dict[int, int]) -> bool:
    """True if all exponents in the factorization are == 1; False otherwise."""
    return all(e == 1 for e in factorization.values())

In [5]:
for k, kfac in factorizations(20).items():
    if is_squarefree(kfac):
        print(f"{k} \t {'*'.join([f'{p}^{e}' for (p, e) in kfac.items()])}")

2 	 2^1
3 	 3^1
5 	 5^1
6 	 2^1*3^1
7 	 7^1
10 	 2^1*5^1
11 	 11^1
13 	 13^1
14 	 2^1*7^1
15 	 3^1*5^1
17 	 17^1
19 	 19^1


A function to count squarefree numbers $\leq n$ can be created using this idea, adding 1 to the count to include the squarefree integer 1.

In [6]:
def Q_factorizations(n: int) -> int:
    """Returns the number of squarefree positive intgers <= n"""
    return 1 + sum(is_squarefree(factorization) for factorization in factorizations(n + 1).values())

In [7]:
time Q_factorizations(10)

CPU times: user 44 µs, sys: 11 µs, total: 55 µs
Wall time: 60.8 µs


7

In [8]:
time Q_factorizations(10**3)

CPU times: user 4.82 ms, sys: 196 µs, total: 5.01 ms
Wall time: 4.93 ms


608

In [9]:
time Q_factorizations(10**6)

CPU times: user 5.54 s, sys: 223 ms, total: 5.77 s
Wall time: 5.79 s


607926

Since our algorithm requires finding lots of prime factorizations, it has a high time and space complexity. This can be observed in the runtimes.

## Sieve method

We can use a sieve to find squarefree integers, similarly to how we can sieve to find primes. A simple sieve of Eratosthenes can be used to find primes (or something fancier if you prefer). Each squarefree number $\leq n$ is divisible by $p^2$ for some $p \leq \sqrt{n}$. First find all primes $\leq \sqrt{n}$, then sieve the interval $1 \ldots n$ to remove all multiples of each of these primes squared to be left with the squarefree integers in the interval.

In [10]:
def primes(n: int):
    block = np.ones(n, dtype=bool)
    block[0] = block[1] = False
    
    for p in range(2, n):
        if block[p]:
            block[p*p::p] = False
    
    return [p for p in range(n) if block[p]]

In [11]:
def Q_sieve(n: int) -> int:
    block = np.ones(n + 1, dtype=bool)
    block[0] = False
    bound = integer_sqrt(n) + 1
    
    for p in primes(bound):
        block[p*p::p*p] = False
    
    return np.count_nonzero(block)

In [12]:
time Q_sieve(10)

CPU times: user 95 µs, sys: 6 µs, total: 101 µs
Wall time: 108 µs


7

In [13]:
time Q_sieve(10**3)

CPU times: user 189 µs, sys: 13 µs, total: 202 µs
Wall time: 176 µs


608

In [14]:
time Q_sieve(10**6)

CPU times: user 3.52 ms, sys: 0 ns, total: 3.52 ms
Wall time: 2.73 ms


607926

In [15]:
time Q_sieve(10**8)

CPU times: user 123 ms, sys: 23.9 ms, total: 147 ms
Wall time: 150 ms


60792694

In [16]:
time Q_sieve(10**9)

CPU times: user 1.18 s, sys: 192 ms, total: 1.37 s
Wall time: 1.37 s


607927124

This method is much faster, but still has $O(n)$ space since we store the entire interval $1 \ldots n$ in memory and $O(n)$ time since we sieve each entry in this interval.


## Segmented Sieve

We can reduce the space complexity by using a segmented sieve. We need to store the primes $\leq \sqrt{n}$, which is roughly $O(\sqrt{n} / \log(n))$ A segmented sieve with block size $B(n)$ would then give an $O(\sqrt{n}/log(n) + B(n))$ space requirement.

Breaking this up with a segmented sieve would not reduce the $O(n)$ time complexity, since we are still sieving all integers in the interval. However, we can reduce the time complexity via other methods.

The idea of a segmented sieve is to break the interval $[1, n]$ into smaller intervals of a given block size, and sieve each of these as we did in the previous algorithm. This allows us to tune the space complexity by adjusting the block size at the cost of more bookkeeping. This adds code complexity as well as to the runtime.

In [17]:
def Q_segmented(n: int) -> int:
    prime_squares = [p**2 for p in primes(integer_sqrt(n))]
    block_size = 2**20
    count = 0
    
    for start in range(1, n + 1, block_size):
        block = np.ones(min(block_size, n - start), dtype=bool)
        for p2 in prime_squares:
            # first multiple of p^2 that is >= start
            multiple = ((start // p2) + (start % p2 > 0))*p2
            block[multiple - start::p2] = False
        count += np.count_nonzero(block)
                
    return count

In [18]:
time Q_segmented(10**8)

CPU times: user 259 ms, sys: 3.6 ms, total: 263 ms
Wall time: 262 ms


60792694

In [19]:
time Q_segmented(10**9)

CPU times: user 5.75 s, sys: 1.07 ms, total: 5.76 s
Wall time: 5.76 s


607927124

## Combinatorial Method I

We first derive a recursive formula for $Q(x)$ by a combinatorial argument.

Partition all the $\lfloor x \rfloor$ positive integers $\leq x$ into subsets $S_k$ for $k = 1, 2, \ldots$ according to their largest square divisor $k^2$. The number of positive integers $\leq x$ having largest square divisor $k^2$ is then $|S_k| = Q(x / k^2)$. It follows that

$$
    \lfloor x \rfloor = \sum_{k \geq 1} |S_k| =  \sum_{1 \leq k \leq \sqrt{x}} Q\left(\frac{x}{k^2}\right).
$$

Rearranging gives a recursive formula for $Q(x)$

$$
    Q(x) = \lfloor x \rfloor - \sum_{2 \leq k \leq \sqrt{x}} Q \left( \frac{x}{k^2} \right).
$$

This can be used to implement a fast method to compute $Q(x)$ by storing previously computed values, a technique sometimes called *memoization* or just simply *caching* the recursion. Notice that we insert initial value $Q(1) = 1$ into the cache so that the recursion terminates.

In [20]:
def Q_memo(x, cache={1: 1}):
    if x not in cache:
        cache[x] = x - sum(Q_memo(x//k**2) for k in range(2, integer_sqrt(x) + 1))
    return cache[x]

In [21]:
time Q_memo(10**6)

CPU times: user 5.62 ms, sys: 83 µs, total: 5.71 ms
Wall time: 5.73 ms


607926

In [22]:
time Q_memo(10**8)

CPU times: user 64.3 ms, sys: 222 µs, total: 64.6 ms
Wall time: 76 ms


60792694

In [23]:
time Q_memo(10**9)

CPU times: user 225 ms, sys: 42 µs, total: 225 ms
Wall time: 225 ms


607927124

In [24]:
time Q_memo(10**12)

CPU times: user 9.96 s, sys: 3.86 ms, total: 9.97 s
Wall time: 9.97 s


607927102274

In [25]:
def Q_moebius(n):
    bound = integer_sqrt(n) + 1
    moebius = np.ones(bound, dtype=int)
    
    for p in primes(bound):
        moebius[p::p] *= -1
        moebius[p*p::p*p] = 0
    
    return sum(moebius[d]*(n//d**2) for d in range(1, bound))

In [26]:
time Q_moebius(10**12)

CPU times: user 3.26 s, sys: 4.15 ms, total: 3.26 s
Wall time: 3.26 s


607927102274